In [1]:
!pip install markovify

     |████████████████████████████████| 245kB 6.5MB/s 
  Created wheel for markovify: filename=markovify-0.9.0-cp36-none-any.whl size=18475 sha256=976700f7a44b375dd24c902fbccb48d81b2f9daaa6937a8bb4b31b7d7278d0da
  Stored in directory: /root/.cache/pip/wheels/03/59/5b/04c5f27d57977580c1122fa69a69c327dede5a30f9dac54c55
Successfully built markovify


In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import gutenberg
from time import time
import re
import spacy
import markovify
import warnings
warnings.filterwarnings("ignore")

# nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'
conn_str = f'postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}'
query = '''
SELECT *
FROM twitter
'''
twitter = pd.read_sql_query(query, con=conn_str)
twitter.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


1. First, do some data preprocessing to clean up the data as necessary. You can use your solution to the assignment of the Text preprocessing checkpoint.
2. Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Do the generated sentences exhibit the same negative sentiment?
3. Repeat the previous task, but this time, use only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.
4. Now, train your Markov chain model using all of the tweets. Generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [4]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [6]:
def create_doc(sentences):
    return " ".join(sentences)

In [15]:
def create_sents(doc):
    return " ".join([sent.text for sent in doc.sents if len(sent.text) > 1])

# Negative sentiment

In [7]:
negative = twitter[twitter.airline_sentiment == "negative"]

sentences = negative.text
doc = create_doc(sentences)

In [8]:
doc = text_cleaner(doc)

In [9]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(doc)

In [10]:
# Group into sentences
# Use the sentences that have more than one character
doc_sents = create_sents(doc)
doc_sents

'@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse @VirginAmerica and it\'s a really big bad thing about it @VirginAmerica seriously would pay $ a flight for seats that didn\'t have this playing. it\'s really the only bad thing about flying VA @VirginAmerica SFO-PDX schedule is still MIA. @VirginAmerica I flew from NYC to SFO last week and couldn\'t fully sit in my seat due to two large gentleman on either side of me. HELP! @VirginAmerica why are your first fares in May over three times more than other carriers when all seats are available to select??? @VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet @VirginAmerica status match program. I applied and it\'s been three weeks. Called and emailed with no response. @VirginAmerica What happened ur vegan food options?! At least say on ur site so i know I won\'t be able eat an

> generate the transition probabilities

In [12]:
tweet_generator = markovify.Text(doc_sents, state_size = 3)

> model is trained; generate random sentences from this model

In [13]:
# Three randomly generated sentences
for i in range(3):
    print(tweet_generator.make_sentence())

# Three randomly generated sentences of no more than 100 characters
for i in range(3):
    print(tweet_generator.make_short_sentence(100))

One is in 1B on flt now to IAH .@united you should really explain customer service to change my flight.
@VirginAmerica my group got their Cancelled Flightlation fees waived but I can't find any Late Flight evening flights.
@united you had us stuck on the tarmac during UA1116.
@Southwestair is this account even aware of the flight details, thanks.
@USAirways flights keep getting delayed and Cancelled Flighted my original reservation.
@AmericanAir We have been stuck in CLT all day.


# Positive Sentiment

In [14]:
positive = twitter[twitter.airline_sentiment == "positive"]
positive.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
1,1,570301130888122368,positive,0.3486,None,0.0,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
6,6,570300616901320704,positive,0.6745,None,0.0,Virgin America,None,cjmcginnis,None,0,"@VirginAmerica yes, nearly every time I fly VX...",None,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
8,8,570299953286942721,positive,0.6559,None,NaN,Virgin America,None,dhepburn,None,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",None,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,9,570295459631263746,positive,1.0000,None,NaN,Virgin America,None,YupitsTate,None,0,"@VirginAmerica it was amazing, and arrived an ...",None,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)
11,11,570289724453216256,positive,1.0000,None,NaN,Virgin America,None,HyperCamiLax,None,0,@VirginAmerica I &lt;3 pretty graphics. so muc...,None,2015-02-24 10:30:40 -0800,NYC,America/New_York


In [20]:
sentences = positive.text
doc = create_doc(sentences)
doc = text_cleaner(doc)


In [21]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(doc)

In [23]:
doc_sents = create_sents(doc)
doc_sents

'@VirginAmerica plus you\'ve added commercials to the experience... tacky. @VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :) @virginamerica Well, I didn\'t…but NOW I DO! :-D @VirginAmerica it was amazing, and arrived an hour early. You\'re too good to me. @VirginAmerica I &lt; pretty graphics. so much better than minimal iconography. :D @VirginAmerica This is such a great deal! Already thinking about my 2nd trip to @Australia &amp; I haven\'t even gone on my 1st trip yet! ; p @VirginAmerica @virginmedia I\'m flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn @VirginAmerica Thanks! @VirginAmerica So excited for my first cross country flight LAX to MCO I\'ve heard nothing but great things about Virgin America. # 29DaysToGo I ❤️ flying @VirginAmerica. ☺️👍 @VirginAmerica you know what would be amazingly awesome? BOS-FLL PLEASE!!!!!!! I want to fly with only you. @VirginAmerica I love this graphic. http:

In [25]:
tweet_generator = markovify.Text(doc_sents, state_size = 3)

In [26]:
# Three randomly generated sentences
for i in range(3):
    print(tweet_generator.make_sentence())

# Three randomly generated sentences of no more than 100 characters
for i in range(3):
    print(tweet_generator.make_short_sentence(100))

None
None
@SouthwestAir @DeltaPoints hey at least you guys are the best airline!
😊☕📲✈ @SouthwestAir Great job!! Looking forward to my flight to tomorrow.
@JetBlue thank you for replying.
@SouthwestAir thanks for the reply.


# All tweets

In [27]:
sentences = twitter.text
# create a doc from the twitter text
doc = create_doc(sentences)
# clean the doc text
doc = text_cleaner(doc)
# process the cleaned doc
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(doc)
# create sents var from processed doc
doc_sents = create_sents(doc)

# train the tweet generator
tweet_generator = markovify.Text(doc_sents, state_size = 3)

# generate random tweets
# Three randomly generated sentences
for i in range(3):
    print(tweet_generator.make_sentence())

# Three randomly generated sentences of no more than 100 characters
for i in range(3):
    print(tweet_generator.make_short_sentence(100))

# SouthwestRally #BWI #bestemployees #swaculture #swapic @SouthwestAir I tried for 5hrs to contact SW!
@USAirways hour at the gate when I get back home.
@AmericanAir you're airline is ridiculous @AmericanAir I don't think you can assist.
@JetBlue what are the odds my bag was &amp; each gave me different info.
@SouthwestAir traveling 2bwi nMaryland 2morrw any chance the flight I was placed on a carousel.
Took off from Miami, then had to return to BOS the first time.
